In [1]:
using PerlaTonettiWaugh, Parameters

┌ Info: Precompiling PerlaTonettiWaugh [d2cacc76-b06b-11e8-362b-f18d03a16405]
└ @ Base loading.jl:1273
┌ Warning: Module BandedMatrices with build ID 176192222987157 is missing from the cache.
│ This may mean BandedMatrices [aae01518-5342-5314-be14-df237901396f] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1011
┌ Warning: Module BandedMatrices with build ID 176192222987157 is missing from the cache.
│ This may mean BandedMatrices [aae01518-5342-5314-be14-df237901396f] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1011
┌ Warning: Replacing module `BandedMatrices`
└ @ Base loading.jl:942


### Baseline Steady State

In [2]:
d_baseline = 3.0426 # d_0
params = parameter_defaults(γ = 1)
settings = settings_defaults();
params_baseline = merge(params, (d = d_baseline,));

In [3]:
stationary_baseline = stationary_algebraic(params_baseline, settings);
println("g: $(stationary_baseline.g)")
println("z_hat: $(stationary_baseline.z_hat)")
println("Omega: $(stationary_baseline.Ω)")
println("c: $(stationary_baseline.c)")
println("U_bar: $(stationary_baseline.U_bar)")
println("lambda: $(stationary_baseline.λ_ii)")
println("y: $(stationary_baseline.y)");

g: 0.006741737930000616
z_hat: 1.9775524589786164
Omega: 0.96128722109579
c: 1.0332687191739103
U_bar: 16.10681381780279
lambda: 0.8956061037770533
y: 1.0332687191739103


### Counterfactual Steady State

In [4]:
d_counterfactual = 1 + 0.99*(d_baseline - 1);
params_counterfactual = merge(params, (d = d_counterfactual,))
stationary_counterfactual = stationary_algebraic(params_counterfactual, settings);

In [5]:
println("g: $(stationary_counterfactual.g)")
println("z_hat: $(stationary_counterfactual.z_hat)")
println("Omega: $(stationary_counterfactual.Ω)")
println("c: $(stationary_counterfactual.c)")
println("U_bar: $(stationary_counterfactual.U_bar)")
println("lambda: $(stationary_counterfactual.λ_ii)")
println("y: $(stationary_counterfactual.y)")
ACR_full = 100*(-1/params.θ)*log(stationary_counterfactual.λ_ii/stationary_baseline.λ_ii)
println("ACR_full: $(ACR_full)");
@show d_U_bar = stationary_counterfactual.U_bar - stationary_baseline.U_bar;

g: 0.006892833763804623
z_hat: 1.9638000950768109
Omega: 0.9568038245589312
c: 1.0322592014126297
U_bar: 16.388219485078398
lambda: 0.8923478897087848
y: 1.0322592014126297
ACR_full: 0.07286641939445852
d_U_bar = stationary_counterfactual.U_bar - stationary_baseline.U_bar = 0.2814056672756067


### Decomposition 1: $\frac{\partial c}{\partial d}$ (change in consumption when changing trade cost, holding fixed Omega, zhat, g)

In [6]:
sol = steady_state_from_g(stationary_baseline.g, stationary_baseline.z_hat, stationary_baseline.Ω, params_counterfactual, settings);

In [7]:
println("c: $(sol.c)")
println("z_hat: $(sol.z_hat)")
println("Omega: $(sol.Ω)")
println("lambda: $(sol.λ_ii)")
println("U_bar: $(sol.U_bar)")
U_bar_partial = sol.U_bar;
ACR_partial = 100*(-1/params.θ)*log(sol.λ_ii/stationary_baseline.λ_ii)
dc = sol.c - stationary_baseline.c
dc_frac = dc/ stationary_baseline.c
println("ACR_partial: $(ACR_partial)")
println("dc: $(dc)")
println("dc_frac: $(dc_frac)")
@show d_U_1 = 1/(stationary_baseline.c * params.ρ) # constant
@show d_U_bar - d_U_1  * dc
@show d_U_Comp1_frac=100*(d_U_bar-d_U_1*dc)/d_U_bar;

c: 1.0340003467567838
z_hat: 1.9775524589786164
Omega: 0.96128722109579
lambda: 0.8942299529521637
U_bar: 16.139735697090448
ACR_partial: 0.03074372884116476
dc: 0.0007316275828734842
dc_frac: 0.0007080709686618737
d_U_1 = 1 / (stationary_baseline.c * params.ρ) = 45.01406753527041
d_U_bar - d_U_1 * dc = 0.248472133849473
d_U_Comp1_frac = (100 * (d_U_bar - d_U_1 * dc)) / d_U_bar = 88.2967767689345


### Decomposition 2: $\frac{\partial c}{\partial g}$ (change in consumption when changing g, holding fixed Omega, zhat, d)

In [8]:
sol = steady_state_from_g(1.01*stationary_baseline.g, stationary_baseline.z_hat, stationary_baseline.Ω, params_baseline, settings);

In [9]:
println("c: $(sol.c)")
println("z_hat: $(sol.z_hat)")
println("Omega: $(sol.Ω)")
println("lambda: $(sol.λ_ii)")
println("U_bar: $(sol.U_bar)")
U_bar_partial = sol.U_bar;
ACR_partial = 100*(-1/params.θ)*log(sol.λ_ii/stationary_baseline.λ_ii)
println("ACR_partial: $(ACR_partial)")
@show dc_frac = (sol.c - stationary_baseline.c)/ stationary_baseline.c;

c: 1.0328334088429962
z_hat: 1.9775524589786164
Omega: 0.96128722109579
lambda: 0.8956061037770533
U_bar: 16.233060742705174
ACR_partial: -0.0
dc_frac = (sol.c - stationary_baseline.c) / stationary_baseline.c = -0.0004212944056431588


### Decomposition 3: $\frac{\partial g}{\partial d}$ (change in g when changing d, holding fixed Omega, zhat, and c)

In [10]:
sol = steady_state_from_c(stationary_baseline.c, stationary_baseline.z_hat, stationary_baseline.Ω, params_counterfactual, settings);

In [11]:
println("g: $(sol.g)")
println("c: $(sol.c)")
println("z_hat: $(sol.z_hat)")
println("Omega: $(sol.Ω)")
println("lambda: $(sol.λ_ii)")
println("U_bar: $(sol.U_bar)")
U_bar_partial = sol.U_bar;
ACR_partial = 100*(-1/params.θ)*log(sol.λ_ii/stationary_baseline.λ_ii)
println("ACR_partial: $(ACR_partial)")
@show dc_frac = (sol.c - stationary_baseline.c)/ stationary_baseline.c
@show dg_frac = (sol.g - stationary_baseline.g) / stationary_baseline.g;

g: 0.006854966379423493
c: 1.0332687191739103
z_hat: 1.9775524589786164
Omega: 0.96128722109579
lambda: 0.8942299529521637
U_bar: 16.35176449259539
ACR_partial: 0.03074372884116476
dc_frac = (sol.c - stationary_baseline.c) / stationary_baseline.c = 0.0
dg_frac = (sol.g - stationary_baseline.g) / stationary_baseline.g = 0.0167951425283105


### Decomposition 4: $\frac{\partial g}{\partial c}$ (change in g when changing c, holding fixed Omega, zhat, and d)

In [12]:
sol = steady_state_from_c(stationary_baseline.c*1.01, stationary_baseline.z_hat, stationary_baseline.Ω, params_baseline, settings);

In [13]:
println("g: $(sol.g)")
println("c: $(sol.c)")
println("z_hat: $(sol.z_hat)")
println("Omega: $(sol.Ω)")
println("lambda: $(sol.λ_ii)")
println("U_bar: $(sol.U_bar)")
U_bar_partial = sol.U_bar;
ACR_partial = 100*(-1/params.θ)*log(sol.λ_ii/stationary_baseline.λ_ii)
println("ACR_partial: $(ACR_partial)")
@show dc_frac = (sol.c - stationary_baseline.c)/ stationary_baseline.c
@show dg_frac = (sol.g - stationary_baseline.g) / stationary_baseline.g;

g: 0.005141494053106584
c: 1.0436014063656494
z_hat: 1.9775524589786164
Omega: 0.96128722109579
lambda: 0.8956061037770533
U_bar: 13.10776186853093
ACR_partial: -0.0
dc_frac = (sol.c - stationary_baseline.c) / stationary_baseline.c = 0.009999999999999974
dg_frac = (sol.g - stationary_baseline.g) / stationary_baseline.g = -0.23736370258075065


### Total Derivative

In [14]:
D = total_derivative(params_baseline);

In [15]:
@unpack U_1, U_2, ∂_c_d, ∂_c_g, ∂_g_c, ∂_g_d, d_U_d, total_decomp, check, planner_0, planner_0_frac, decomp_1_frac, decomp_2_frac, decomp_3_frac, decomp_4_frac = D;

In [16]:
@show U_1
@show U_2
@show ∂_c_d
@show ∂_c_g
@show ∂_g_d
@show ∂_g_c
@show d_U_d
@show total_decomp
@show check
@show planner_0
@show planner_0_frac
@show decomp_1_frac
@show decomp_2_frac
@show decomp_3_frac
@show decomp_4_frac;

U_1 = 45.01406753527041
U_2 = 2163.331530557058
∂_c_d = 0.0007316275828734842
∂_c_g = -0.0004353103309140405
∂_g_d = 0.00011322844942287733
∂_g_c = -0.0016002438768940317
d_U_d = 0.2814056672756067
total_decomp = 0.27534919867048796
check = -0.006056468605118714
planner_0 = 0.24494845607109328
planner_0_frac = 0.8895920425910684
decomp_1_frac = 0.11960642553220364
decomp_2_frac = -8.057846229837256e-6
decomp_3_frac = 0.8896001004372982
decomp_4_frac = -0.009198468123271873
